In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets transformers --quiet

     |████████████████████████████████| 452 kB 32.2 MB/s 
     |████████████████████████████████| 5.8 MB 64.6 MB/s 
     |████████████████████████████████| 132 kB 79.4 MB/s 
     |████████████████████████████████| 212 kB 79.1 MB/s 
     |████████████████████████████████| 182 kB 77.6 MB/s 
     |████████████████████████████████| 127 kB 70.0 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 


In [3]:
import json
import time
from pprint import pprint
import gzip
import os
import torch
import pandas as pd
from collections import Counter
from tqdm import tqdm
from datasets import load_dataset

In [ ]:
# !rm -rf /content/distilbert-base-uncased-finetuned_focus_swag

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !rm -rf /content/content

In [5]:
from datasets import load_dataset, load_metric, Dataset

In [6]:
train_csv = "/content/drive/MyDrive/UMBC/train_swag_easy_ground_knowledge.csv"
val_csv = "/content/drive/MyDrive/UMBC/val_swag_easy_ground_knowledge.csv"
test_csv = "/content/drive/MyDrive/UMBC/test_swag_easy_ground_knowledge.csv"

In [7]:
dataset = load_dataset('csv', data_files={'test': test_csv,
                                          'validation': val_csv, 
                                          'train': train_csv})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-22b3d46bed5fa690/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 2

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
persona_candidate_columns = ["persona1", "persona2", "persona3", "persona4", "persona5", "persona6"] #persona6 for none of these 
# persona_grounding_columns = ["persona_grounding1", "persona_grounding2", "persona_grounding3", "persona_grounding4", "persona_grounding5"] #persona_grounding6 for none of these 

In [11]:
def preprocess_function(examples):

  first_sentences = [[f"{query} {hit_knowledge}"]*6 for query, hit_knowledge in zip(examples['query'], examples['ground_knowledge'])]
  second_sentences = [[examples[persona_candidate_column][i] for persona_candidate_column in persona_candidate_columns]for i, _ in enumerate(examples['dialogID'])]
  first_sentences = sum(first_sentences, [])
  second_sentences = sum(second_sentences, [])

  tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
  return {k: [v[i:i+6] for i in range(0, len(v), 6)] for k, v in tokenized_examples.items()}


In [12]:
dataset_encoded = dataset.map(preprocess_function, batched=True)
# test_encoded = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/56 [00:00<?, ?ba/s]

In [13]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels)
        # batch["labels"] = torch.tensor(labels, dtype=torch.float64)
        return batch

In [14]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

# import numpy as np

# def compute_metrics(eval_predictions):
#     predictions, label_ids = eval_predictions
#     preds = np.argmax(predictions, axis=1)
#     return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [15]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint, num_labels=1)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultipleChoice: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']

In [16]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned_focus_swag",
    evaluation_strategy = "epoch",
    learning_rate=7e-7,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit = 1,
    # push_to_hub=True,
)

In [17]:
dataset_encoded

DatasetDict({
    test: Dataset({
        features: ['dialogID', 'utterance', 'hit_knowledge', 'query', 'label', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona6', 'ground_knowledge', 'input_ids', 'attention_mask'],
        num_rows: 8644
    })
    validation: Dataset({
        features: ['dialogID', 'utterance', 'hit_knowledge', 'query', 'label', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona6', 'ground_knowledge', 'input_ids', 'attention_mask'],
        num_rows: 8641
    })
    train: Dataset({
        features: ['dialogID', 'utterance', 'hit_knowledge', 'query', 'label', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona6', 'ground_knowledge', 'input_ids', 'attention_mask'],
        num_rows: 55658
    })
})

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset_encoded['validation'],
    eval_dataset=dataset_encoded['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMultipleChoice.forward` and have been ignored: persona5, persona3, ground_knowledge, utterance, persona2, persona4, dialogID, query, hit_knowledge, persona6, persona1. If persona5, persona3, ground_knowledge, utterance, persona2, persona4, dialogID, query, hit_knowledge, persona6, persona1 are not expected by `DistilBertForMultipleChoice.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8641
  Num Epochs = 4
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps

Epoch,Training Loss,Validation Loss,Accuracy
1,0.736000,0.577209,0.854928
2,0.422700,0.438456,0.887089
3,0.552200,0.473607,0.892411


Saving model checkpoint to distilbert-base-uncased-finetuned_focus_swag/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-500/special_tokens_map.json
Saving model checkpoint to distilbert-base-uncased-finetuned_focus_swag/checkpoint-1000
Configuration saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-1000/config.json
Model weights saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned_focus_swag/checkpoint-1000/s

In [ ]:
trainer.train()

In [ ]:
# !zip -r /content/drive/MyDrive/UMBC/easy_weights_val_bs2.zip /content/distilbert-base-uncased-finetuned_focus_swag

  adding: content/distilbert-base-uncased-finetuned_focus_swag/ (stored 0%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/runs/ (stored 0%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/runs/Dec20_15-32-26_caebeca242b3/ (stored 0%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/runs/Dec20_15-32-26_caebeca242b3/1671550358.3561466/ (stored 0%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/runs/Dec20_15-32-26_caebeca242b3/1671550358.3561466/events.out.tfevents.1671550358.caebeca242b3.21550.1 (deflated 62%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/runs/Dec20_15-32-26_caebeca242b3/events.out.tfevents.1671550358.caebeca242b3.21550.0 (deflated 59%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-8500/ (stored 0%)
  adding: content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-8500/tokenizer_config.json (deflated 41%)
  adding: content/distilbert-base-uncased-finetuned_f

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained("/content/content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-2500")
model.to('cuda')

loading configuration file /content/content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-2500/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-2500",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMultipleChoice"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file /content/content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-2500/pytorch_model.bin
All model checkpoint weights were used when i

DistilBertForMultipleChoice(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (li

In [ ]:
dataset_encoded['test']

Dataset({
    features: ['dialogID', 'utterance', 'hit_knowledge', 'query', 'label', 'persona1', 'persona2', 'persona3', 'persona4', 'persona5', 'persona6', 'input_ids', 'attention_mask'],
    num_rows: 8644
})

In [ ]:
for data in dataset_encoded['test']:
  print(data)
  break

{'dialogID': '4P3EWN6HNTMN', 'utterance': 1, 'hit_knowledge': 'The Theatre of Dionysus (or Theatre of Dionysos, gr: Θέατρο του Διονύσου) is an ancient Greek theatre in Athens. It is built on the south slope of the Akropolis hill,originally part of the sanctuary of Dionysos Eleuthereus (Dionysus the Liberator).', 'query': 'Tell me more about the location of the theatre?', 'label': 5, 'persona1': 'I love Nile river.', 'persona2': 'I wish to follow Joanne Anes.', 'persona3': 'I want to participate in car racing.', 'persona4': 'I love drawing trees.', 'persona5': 'I like the sense of community in the rural areas.', 'persona6': ' ', 'input_ids': [[101, 2425, 2033, 2062, 2055, 1996, 3295, 1997, 1996, 3004, 1029, 1996, 3004, 1997, 19542, 7274, 2271, 1006, 2030, 3004, 1997, 19542, 7274, 2891, 1010, 24665, 1024, 1162, 29723, 14608, 29734, 29732, 29730, 1174, 26789, 1158, 18199, 29730, 16177, 29735, 29733, 26789, 1007, 2003, 2019, 3418, 3306, 3004, 1999, 7571, 1012, 2009, 2003, 2328, 2006, 1996,

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("/content/distilbert-base-uncased-finetuned_focus_swag/checkpoint-8500")


loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_positio

In [ ]:
from pprint import pprint

In [ ]:
num_rows = 8643
indexes = np.random.choice(num_rows, 10)
# indexes = [6265,  466, 4426, 5578, 8322]
# [6265,  466, 4426, 5578, 8322]
indexes = [7270,  860, 5390, 5191, 5734, 6265,  466, 4426, 5578, 8322]

In [ ]:
idx = 0
for idx in indexes:
  data = dataset_encoded['test'][int(idx)]
  prompt = data['hit_knowledge']
  persona1 = data['persona1']
  persona2 = data['persona2']
  persona3 = data['persona3']
  persona4 = data['persona4']
  persona5 = data['persona5']
  persona6 = data['persona6']
  label = data['label']

  inputs = tokenizer(text=[[prompt, persona1], [prompt, persona2], [prompt, persona3],
                      [prompt, persona4], [prompt, persona5], [prompt, persona6]],
                      return_tensors="pt", padding=True).to("cuda")
  outputs = model(**{k: v.unsqueeze(0)  for k, v in inputs.items() if k != "token_type_ids"})
  logits = outputs.logits
  predicted_class = logits.argmax().item()
  pprint([prompt, persona1, persona2, persona3, persona4, persona5, persona6, label])
  pprint({"predicted": predicted_class})
  print("*"*100)

['The hostels are named after the principal rivers of India and the campus '
 'buses used to be named after mountains, resulting in an epigram about IIT '
 'Madras that it is the only place where the mountains move and the rivers '
 'remain still. With the discontinuance of naming of buses and the mess halls '
 'being named Vindhya and Himalaya, this epigram is no longer meaningful.',
 'I would like to visit Chennai.',
 'I like visiting cathedrals or churches.',
 'I am a fan of Alexandre Dumas.',
 'I want to visit the United States someday.',
 'I want to learn about religions.',
 ' ',
 0]
{'predicted': 0}
****************************************************************************************************
['The area has become a major tourist attraction along the Southeastern coast, '
 'with its primary city, Myrtle Beach, attracting over ten million visitors '
 'each season. [citation needed] It is home to numerous hotels, golf resorts, '
 'and recreational centers, making it popular w

In [ ]:
!unzip /content/drive/MyDrive/UMBC/easy-focus-persona-selection_bs6.zip 